<style>
@font-face {font-family: "B Nazanin"; src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot"); src: url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.eot?#iefix") format("embedded-opentype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff2") format("woff2"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.woff") format("woff"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.ttf") format("truetype"), url("//db.onlinewebfonts.com/t/3671adca6f650c92b83f906e49656986.svg#B Nazanin") format("svg"); }
    </style>
<div dir="rtl" align="center" style ='font-family: "B Nazanin";'>
    <h1>
        تمرین اول درس پردازش زبان‌های طبیعی
    </h1>
    <h3>
        گردآورندگان:<br/>
        ساحل مس‌فروش، سروش تابش، درنا دهقانی
    </h3>
    
</div>

<br/>
<br/>


<div dir="rtl"  style ='font-family: "B Nazanin";'>
    ما در این تمرین ترک "استخراج پرسش و پاسخ" را انتخاب کردیم. در این تمرین قصد داریم از یک جمله فارسی سوالاتی با توجه به روابط علت و معلولی، نقش کلمات در جملات، ارتباط بین واژگان و ... استخراج کنیم. هر سوال به شکل یک دیکشنری پایتون نمایش داده می‌شود که key سوال و value پاسخ به سوال است. گام ابتدایی ما برای پاسخ به این ترک، جستجوی مسائل مشابه بود و از آنجایی که این مسئله هم‌چنان باز است، روش‌های متعددی مبتنی بر Deep Learning یافتیم که چون از مبحث این تمرین خارج بودند، از آنها استفاده نکردیم.
</div>

<div dir="rtl" style ='font-family: "B Nazanin";'>
    <h2> 1. تمیز کردن ورودی </h2>
    در این بخش، گام‌های ابتدایی برای مراحل آتی را برمی‌داریم. هر جمله که به عنوان ورودی ما داده می‌شود، باید تا حد مناسبی تمیز و ساختاریافته شود تا به عنوان ورودی بخش‌های بعد معتبر باشد. کارهایی مانند نرمال‌سازی و ... در این بخش انجام می‌گردد. از توابع تعریف شده در کتابخانه hazm استفاده می‌کنیم.
</div>

In [1]:
!pip install hazm
!pip install camel-tools
!pip install Tashaphyne
!pip install parstdex~=1.1.0

#  نصب کتاب‌خانه مورد نیاز برای اتصال به وب‌سرویس فارس‌نت
! pip install zeep

     |████████████████████████████████| 316 kB 17.0 MB/s 
     |████████████████████████████████| 1.4 MB 65.1 MB/s 
     |████████████████████████████████| 233 kB 55.9 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=edbfc0fad24ba8694c5aaf961edd5d382f284756104ec7cfa1da11d22ca8e515
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154939 sha256=03c0f3faba51792e68659206bc0ae7b4743a98cfbfe805d1075302f64ac207f6
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 102 kB 16.3 MB/s 
     |████████████████████████████████| 418 kB 60.2 MB/s 
    

     |████████████████████████████████| 100 kB 7.7 MB/s 
     |████████████████████████████████| 41 kB 688 kB/s 
     |████████████████████████████████| 6.4 MB 54.9 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
import re
from collections import Counter
import json
import os
from itertools import chain
import collections
import hazm
import random
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import itertools
import functools
import operator
import matplotlib
from zeep import Client
from requests.auth import HTTPBasicAuth
from requests import Session
from zeep.transports import Transport

In [3]:
def sentence_clean(sentence: str):
    normalizer = hazm.Normalizer()
    sentence = normalizer.normalize(sentence)
    return sentence.split('. ')

sentence = 'چون بارون اومده  ، رنگین کمونه. هوا بارونیه. '
print(sentence_clean(sentence))

['چون بارون اومده، رنگین کمونه', 'هوا بارونیه', '']


<div dir="rtl" style ='font-family: "B Nazanin";'>
    <h2> 2. استخراج ساختار سوالات </h2>
    جملاتی که امکان استخراج سوالات از آنها ممکن است، دو نوعند. نوع اول دارای برخی کلمات کلیدی (مانند زیرا، چون و ...) هستند. نوع دوم جملاتی هستند که فاقد اینگونه کلماتند. مثلاً امروز هوا بارانیست: امروز هوا چگونه است؟ بارانی. در طراح این بخش از ماژول <code>cause_effect_extractions</code> ایده گرفتیم.<br>
    <h4> 2.1. استخراج سوال از جملات با کلمات کلیدی </h4>
    در این مرحله، به یافتن کلمات کلیدی در جملاتی که امکان استخراج سوالات از آنها وجود دارد پرداختیم. در یک Google Sheet لیست این کلمات، انواع نحوه به کارگیری آنها در جملات، مثال‌هایی از هر یک و هم‌چنین ساختار سوال و جواب و مثال از آنها را مشخص کردیم. 
    هم‌چنین مثال‌هایی از جملات را به عنوان داده‌ی تست یافتیم و سوال‌هایی از آنها استخراج کردیم.
    <a href="https://docs.google.com/spreadsheets/d/1qsCfRVr5RgXYW2qEVbrP3dq7-l3wNngjPAsXrHN89DQ/edit#gid=0" target="_top">فهرست کلمات کلیدی و جملات تست</a><br/>
    سپس در تابع <code>extract_cause_effect</code> از جمله‌ای که به عنوان ورودی داده می‌شود، با توجه به ساختار جمله که ورودی دیگر این تابع و از انواع پرسش و پاسخ است، رابطه‌ی علت و معلولی که ممکن است در جمله باشد، استخراج می‌گردد.<br>
    در دیکشنری <code>all_regexes</code> نوع و ساختار جملاتی که قابلیت طرح سوال از آنها وجود دارد توسط regex طراحی شده است. هم‌چنین ساختار پرسش و پاسخ آنها نیز وجود دارد.<br>
    اگر ساختار regex در جمله موجود بود، بسته به اینکه کدام بخش جمله در سوال و کدام بخش آن در جواب استفاده شود، سوال و جواب از جمله ساخته شده و در دیکشنری خروجی تابع قرار می‌گیرد.
</div>

In [4]:
all_regexes = {
  "چرایی": {"regex": r"""(?P<effect>.*)[،|؛|](?P<reason> زیرا| چون| به‌دلیل| بدلیل|چون\sکه | به\sدلیل\sاینکه)(?P<cause>.*)|چون(?P<cause2>.*)(،|\s،)(?P<effect2>.*)""", "question": "چرا {effect} ؟", "answer": "زیرا {cause}"}, 
  "سببی": {"regex": r"(?P<effect>.*) (?P<reason> مسبب|دلیل|مسبّب) (?P<cause>.*) [است|شد]|(?P<reason2>مسبب|دلیل) (?P<effect2>.*)[،|؛] (?P<cause2>) است.", "question": "{effect} باعث چه می شود؟ ", "answer": "باعث {cause}"},
  "موجب": {"regex":r"(?P<effect>.*) موجب (?P<cause>.*) می‌شود.*|(?P<effect2>.*) موجبات (?P<cause2>.*) فراهم می‌کند", "question": "چه موجب {effect} می‌شود؟", "answer": "{cause}"},
  "ترین": {"regex":r"(?P<tarin>.*ترین) (?P<cause>.*)[،|؛](?P<effect>.*) است", "question": "{tarin}ترین {cause} چیست؟", "answer": "{effect}"},
  "خاطر": {"regex": r"""(?P<reason> بخاطر|به خاطر) (?P<cause>.*)[،|] (?P<effect>.*)|(?P<effect2>.*) (?P<reason2> بخاطر|به خاطر) (?P<cause2>.*)""", "question": "چرا {effect} ؟", "answer": "به خاطر {cause}"},
  "منجر": {"regex": r"(?P<cause>.*)(?P<reason> منجر به)(?P<effect>.*)می شود.|(?P<reason2>مسبب|دلیل) (?P<effect2>.*)[،|؛] (?P<cause2>) است.", "question": "علت {effect} چیست؟", "answer": "{cause}"},
   "هدف": {"regex":r"هدف (?P<effect>.*) (?P<reason>این\sاست\sکه|این\sاست) (?P<cause>.*)", "question": "هدف {effect} چیست؟", "answer": "{cause}"},
  #  "": {"regex":r"", "question": "", "answer": ""},
}

def extract_cause_effects(text: str, key: str):
    """
    extracting cause and effect parts from text based on key regex. creating a Q&A based on key question format and answer format
    Parameters
    ---------
    text: str
        sentence query used to extract question and answers.
    key: str
        key should be one of all_regexes.keys(). sentence format used to extract Q&A
  
  
    Returns
    -------
    result: dict
        includes question and answer sentences.
    """
    
    
    key_information = all_regexes.get(key, None)
    if not key_information:
        print("you should specify current key information in all_regexes dict")
        return
    regex = key_information.get("regex")
    first_reg = re.compile(regex)
    x = re.search(first_reg, text)
    if x is not None:
        cause = x.group("cause")
        effect = x.group("effect")
        if not cause:
        # text is not in the format of first part of regex. checking for second part
            cause = x.group("cause2")
            effect = x.group("effect2")
            if not cause or not effect:
                # cause and effects are empty! this shouldn't happen!!!
                print("bad input")
                return None
    else:
        # regex didn't found the structure. 
        return None
    print("in func for: ", key)
    q_format = key_information.get("question")
    a_format = key_information.get("answer")  
    result = {"question": sentence_clean(q_format.format(effect=effect)), "answer": sentence_clean(a_format.format(cause=cause))}
    return result

<div dir="rtl" style ='font-family: "B Nazanin";'>
    در تابع <code>extract_question</code> ابتدا روی تمامی ساختارهای موجود که امکان طرح سوال از آنها وجود دارد، پیمایشی انجام می‌گیرد. در هر پیمایش، هر ساختار از <code>all_regexes</code> در جمله‌ی ورودی توسط تابع <code>extract_cause_effects</code> بررسی می‌گردد و در صورت داشتن خروجی معتبر، پرسش و پاسخ مربوطه‌ی آن نمایش داده می‌شود.
</div>

In [5]:
def extract_questions(text: str):
    """
    checking all different sentence formats on text. if one is applied ignore others and return
    Parameters
    ----------
    text: str
        input sentence(s)
    
    Returns
    ------
    result: dict
      question and answer sentences.
    """
    text = sentence_clean(text)
    for t in text:
      if len(t) > 1:
        for key, value in all_regexes.items():
            result = extract_cause_effects(t, key)
            if result:
                print(result)
                return result
    
sentence = 'رنگین کمونه؛ چون بارون اومده. چون بارون اومده؛ رنگین کمونه.'
extract_questions(sentence)

in func for:  چرایی
{'question': ['چرا رنگین کمونه؟'], 'answer': ['زیرا بارون اومده']}


{'answer': ['زیرا بارون اومده'], 'question': ['چرا رنگین کمونه؟']}

<div dir='rtl' style ='font-family: "B Nazanin";'>
    <h4>2.2. استخراج سوال از جملات بدون کلمات کلیدی</h4>
    عملکرد کلی در این بخش بدین صورت است که امکان پرسش با حذف بخشی از جمله ممکن است و به جای بخش حذف شده، ضمیر پرسشی مربوطه جایگزین آن می‌گردد و سوال تشکیل می‌شود. در نتیجه بخش حذف شده از جمله، پاسخ به آن پرسش است.<br>
    مثلاً در جمله‌ی "او کتاب را برداشت."، با حذف "او" که اشاره به فرد دارد و جایگزین کردن "چه کسی" یک سوال ساخته می‌شود. هم‌چنین با حذف "کتاب" که اشاره به شیء دارد و جایگزین کردن "چه چیزی" می‌توان سوال دیگری ساخت.<br>
    چالشی که در این بخش با آن مواجه بودیم، نحوه برخورد با گروه‌های اسمی بود که با حرف اضافه شروع می‌شدند. مثلاً در جمله‌ی "او با آرامش راه رفت." تشخیص اینکه "با" حرف ربط یا حرف اضافه است برای ما مسئله بود. در چنین شرایطی تصمیم گرفتیم به کلمه‌ی پیش از این حرف نگاه کنیم و اگر فعل بود، آن را حرف ربط و در غیر اینصورت حرف اضافه در نظر بگیریم. در این صورت کلمه‌ی پرسشی مناسب برای گروه اسمی "با آرامش"، "چگونه" است. <br>
    برای استفاده از تابع <code>POSTagger</code> از کتابخانه‌ی hazm، به دلیل مشکلاتی که در استفاده‌ی مستقیم از کتابخانه‌ی آن پیش می‌آمد، مجبور شدم از محتوای این تابع به شکلی دستی استفاده کنم در فولدر resources قرار دارد.
</div>

In [6]:
# download "resources" from here: https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
tagger = hazm.POSTagger(model='/content/resources/postagger.model')

<div dir='rtl' style ='font-family: "B Nazanin";'>
  بخش‌های قابل پرسش در جمله به چهار گروه متممی، مفعولی، نهادی و قیدی تقسیم می‌شوند. در تابع <code>extract_structure</code> برای هر یک از این ساختارها، regex مربوطه نوشته شده و به کمک تابع <code>POSTagger</code> کتابخانه‌ی hazm، درخت روابط واژگانی جمله نمایش داده می‌شود.
</div>

In [7]:
questionable_poses = ['MOTAMEMI', 'MAFOULI', 'NAHADI', 'GHEIDI']


def extract_structures(sentence):
    neutral_words = r"N|Ne|PRO|PROe"
    grammar = r"""
      MOTAMEMI:  {<P|Pe><N|Ne>(<CONJ>?<N|Ne|AJ|AJe>)*}
      MAFOULI: {<N|Ne><POSTP>}
      NAHADI:  {<DET>?<NW>((<CONJ><NW>)|<AJe|AJ>)*}
      GHEIDI:  {<ADV>+}
    """
    grammar = grammar.replace('NW', neutral_words)
    cp = nltk.RegexpParser(grammar)
    return cp.parse(sentence)

tree = extract_structures(tagger.tag(hazm.word_tokenize('مجلس اولویتی برای کاهش هزینه اجرایی کشور که عامل تورم در کشور است، ندارند.')))
list(tree.subtrees())

[Tree('S', [Tree('NAHADI', [('مجلس', 'Ne')]), Tree('NAHADI', [('اولویتی', 'N')]), Tree('MOTAMEMI', [('برای', 'Pe'), ('کاهش', 'Ne'), ('هزینه', 'Ne'), ('اجرایی', 'AJe'), ('کشور', 'N'), ('که', 'CONJ'), ('عامل', 'Ne'), ('تورم', 'N')]), Tree('MOTAMEMI', [('در', 'P'), ('کشور', 'N')]), ('است', 'V'), ('،', 'PUNC'), ('ندارند', 'V'), ('.', 'PUNC')]),
 Tree('NAHADI', [('مجلس', 'Ne')]),
 Tree('NAHADI', [('اولویتی', 'N')]),
 Tree('MOTAMEMI', [('برای', 'Pe'), ('کاهش', 'Ne'), ('هزینه', 'Ne'), ('اجرایی', 'AJe'), ('کشور', 'N'), ('که', 'CONJ'), ('عامل', 'Ne'), ('تورم', 'N')]),
 Tree('MOTAMEMI', [('در', 'P'), ('کشور', 'N')])]

<div dir='rtl' style ='font-family: "B Nazanin";'>
  برای تشخیص اسنادی بودن یا نبودن یک جمله از تابع <code>is_linking_centence</code> استفاده می‌کنیم و بررسی اسنادی بودن فعل جمله توسط تابع <code>is_linking_verb</code> انجام می‌گردد.
</div>

In [8]:
LINKING_VERBS = {'است', 'بود', 'شد', 'گشت', 'گردید', 'هست'}


def is_linking_verb(verb):
    verb = set(hazm.Lemmatizer().lemmatize(verb).split('#'))
    return bool(LINKING_VERBS.intersection(verb))


def is_linking_sentence(sentence_tokens):
    words = tagger.tag(sentence_tokens)
    verb_types = map(is_linking_verb, map(lambda x: x[0], filter(lambda x: x[1] == 'V', words)))
    return functools.reduce(operator.or_, verb_types, False)

is_linking_sentence(hazm.word_tokenize('این خانه زیبا است.'))

True

<div dir="rtl" style ='font-family: "B Nazanin";'>
  برای تشخیص نوع بخش حذف شده از جمله و جایگزینی ضمیر پرسشی مناسب، از کتابخانه‌ی wordnet فارسی یا فارس‌نت استفاده کردیم. ابتدا ثبت نام کرده و مراحل لازم برای اتصال به آن را انجام دادیم.
</div>

In [9]:
# address of FarsNet's web services
wsdl_sense_service = 'http://nlp.sbu.ac.ir:8180/WebAPI/services/SenseService?WSDL'
wsdl_synset_service = 'http://nlp.sbu.ac.ir:8180/WebAPI/services/SynsetService?WSDL'


# username and token needed for authentication. You can get this token by signing up on http://farsnet.nlp.sbu.ac.ir
# users = [{"username": ""}, {"username"}]
username = 'SahelMesforoush'
token = 'd428e755-3b91-11eb-8a1e-080027d731c1'

# connecting client
session = Session()
session.auth = HTTPBasicAuth(username, token)
client_sense = Client(wsdl_sense_service, transport=Transport(session=session))
client_synset = Client(wsdl_synset_service, transport=Transport(session=session))

ConnectionError: ignored

<div dir="rtl" style ='font-family: "B Nazanin";'>
  برای انتخاب ضمیر پرسشی مناسب، لازم است نوع واژه را از نظر اشاره داشتن به فرد، مکان، زمان و یا حالت بدانیم. به کمک تابع <code>getSensesByWord</code> می‌توان درباره‌ی هر واژه اطلاعاتی به دست آورد. فیلد nounCategory اطلاعاتی در مورد خاص یا غیر خاص بودن اسم  می‌دهد. مثلاً برای نام "فاطمه" این فیلد برابر با <code>Specific</code> و برای "درخت" برابر با <code>None</code> است. فیلد می‌تواند در مورد نوع نام، مانند انسان بودن اطلاعاتی به ما دهد.<br>
  به کمک تابع <code>getSynsetsByWord</code> و فیلد semanticCategory از آن نیز می‌توان در مورد نوع قیود از نظر زمانی و مکانی و غیره بودن را تشخیص داد. مثال‌هایی از این دو تابع را می‌توان مشاهده کرد:
</div>

In [ ]:
request = {"searchKeyword": "درخت", "userKey": "d428e755-3b91-11eb-8a1e-080027d731c1", "searchStyle": "EXACT"}
sense = client_sense.service.getSensesByWord(**request)
print(sense)

request = {"searchKeyword": "فاطمه", "userKey": "d428e755-3b91-11eb-8a1e-080027d731c1", "searchStyle": "EXACT"}
sense = client_sense.service.getSensesByWord(**request)
print(sense)

request = {"searchKeyword": "دیروز", "userKey": "d428e755-3b91-11eb-8a1e-080027d731c1", "searchStyle": "EXACT"}
sense = client_synset.service.getSynsetsByWord(**request)
print(sense)

request = {"searchKeyword": "آنجا", "userKey": "d428e755-3b91-11eb-8a1e-080027d731c1", "searchStyle": "EXACT"}
sense = client_synset.service.getSynsetsByWord(**request)
print(sense)

<div dir="rtl" style ='font-family: "B Nazanin";'>
بدین ترتیب به کمک این توابع می‌توان نوع بسیاری از واژگان یا گروه‌های اسمی را تشخیص داد و متناسب اسم خاص یا غیرخاص، انسان یا غیر انسان، حالت، ارتباط، مکان ، زمان و ...، ضمیر پرسشی مناسب تعیین کرد.
</div>

<div dir="rtl" style ='font-family: "B Nazanin";'>
    <h2> 3. صحت‌سنجی </h2>
    ابتدا در فایل <code>question_generator.json</code> تعدادی مثال و خروجی‌های مورد توقع از آنها برای پرسش و پاسخ نوشته شد. سپس توسط کلاس <code>BaseTest</code> توابعی برای صحت‌سنجی عملکرد بخش‌های پیشین در یافتن و استخراج پرسش و پاسخ از جملات نوشته شد. این کلاس و دو تابع <code>get_testcases</code> و <code>run_test</code> از کتابخانه‌ی parsi.io گرفته شده‌اند.
</div>

In [ ]:
class BaseTest:

    def get_testcases(self, addr):
        f = open(os.getcwd()+addr, 'r')
        test_cases = json.load(f)
        f.close()
        return test_cases

    def run_test(self, obj, addr):
        errors = []
        test_cases = self.get_testcases(addr)
        for i in test_cases:
            your_answer = obj.run(i['input'])
            correct_answer = i['outputs']
            d = {k: (your_answer[k], correct_answer[k]) for k in your_answer if k in correct_answer and your_answer[k] != correct_answer[k]}
            for j in d:
                errors.append('Input {0}: your answer is {1} correct answer is {2}'.format(i['id'], d[j][0], d[j][1]))
        assert not errors, 'errors occured:\n{}'.format('\n'.join(errors))

    def test_json(self):
        test_cases = self.get_testcases('/question_generator.json')
        for i in test_cases:
            your_answer = extract_questions(i.get('input'))
            correct_answer = i.get('output')
            print("Testing {0}: ".format(str(i['id']) ))
            print("input: ", i.get("input"))
            print("correct answer: ", correct_answer)
            print("method answer: ", your_answer)
            print("***********************************")

tester = BaseTest()
tester.test_json()

in func for:  چرایی
{'question': ['چرا شما با ذکرهای زیر به خوبی آشنایید؟'], 'answer': ['زیرا در هر نماز آنها را تکرار میکنید']}
Testing 1: 
input:  شما با ذکرهای زیر به خوبی آشنایید؛ زیرا در هر نماز آنها را تکرار میکنید
correct answer:  {'question': 'چرا شما با ذکرهای زیر به خوبی آشنایید?', 'answer': 'زیرا در هر نماز آنها را تکرار میکنید'}
method answer:  {'question': ['چرا شما با ذکرهای زیر به خوبی آشنایید؟'], 'answer': ['زیرا در هر نماز آنها را تکرار میکنید']}
***********************************
in func for:  چرایی
{'question': ['چرا رنگین کمونه؟'], 'answer': ['زیرا بارون اومده.']}
Testing 2: 
input:  رنگین کمونه؛ چون بارون اومده.
correct answer:  {'question': 'چرا رنگین کمونه؟', 'answer': 'چون بارون اومده'}
method answer:  {'question': ['چرا رنگین کمونه؟'], 'answer': ['زیرا بارون اومده.']}
***********************************
Testing 3: 
input:  بی احتیاطی مسبّب تصادف شد.
correct answer:  {'question': 'مسبّب تصادف چیست؟', 'answer': 'بی احتیاطی'}
method answer:  None
****************